In [ ]:
%reset -f
import time
import pickle
import gym
from stable_baselines3 import DDPG
from stable_baselines3.common.env_checker import check_env
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import derivatives as d
import forward_helpers as fh
from chameleon import Chameleon

## Doing the RL

In [ ]:
env = Chameleon(E=.10, target_pos=0.9, dt=1e-6)
model = DDPG("MlpPolicy", env, verbose=0)

In [ ]:
model.learn(total_timesteps=int(5e3))

In [ ]:
%%time
active_stress = 10 * np.linspace(1, 10, 50)
# env.step(np.array([.91,.92,.93], dtype=np.float32))
fh.forward_simulate(env, active_stress, sim_steps=10_000)
# diffs = np.diff(env.pos_f)
# idxs = np.where(diffs < 0)

In [ ]:
env.pos_f

In [ ]:
len(env.position_history)

In [ ]:
diffs = np.diff(env.pos_f)
idxs = np.where(diffs < 0)
idxs

In [ ]:
# history = []
# for i in range(len(chameleon.position_history)):
#     x_p = chameleon.position_history[i]
#     y = np.zeros(chameleon.n_elems)
#     ar = np.vstack((x_p, y)).T
#     history.append(ar)

# with open("history.txt", "wb") as fp:
#     pickle.dump(history, fp)